In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
from fastai import *
from fastai.vision import *

# !pip install gpustat

%matplotlib inline

%cd /content/drive/My\ Drive/alfalfa-convnets

PATH = os.path.abspath('.')

seed = 12345

/content/drive/My Drive/alfalfa-convnets


In [3]:
sys.path.append(f'{PATH}/src')

from mxresnet import *
from ranger import Ranger

Mish activation loaded...
Mish activation loaded...


# Train

In [5]:
lr = "6e-2"
bs = "64"
n_epochs = "40"
val_pct = f"{541/1197}"

%run -i "/content/drive/My Drive/alfalfa-convnets/src/train_xGASS.py" \
    --environment=True --model=mxresnet34 --val-pct={val_pct}\
    --lr={lr} --bs={bs} --n_epochs={n_epochs} \
    --precision=full --seed={seed} \
    --save="best_xGASS_environment-34"

Loaded xGASS catalog of length 1179


epoch,train_loss,valid_loss,time
0,0.837212,0.791628,00:11
1,0.653153,0.644269,00:11
2,0.588190,0.661267,00:11
3,0.534476,0.671565,00:11
4,0.499860,0.808309,00:11
5,0.473635,0.753685,00:11
6,0.453480,1.206466,00:11
7,0.439931,0.519092,00:11
8,0.426938,0.407736,00:11
9,0.420338,0.443241,00:11


Better model found at epoch 0 with valid_loss value: 0.7916277050971985.
Better model found at epoch 1 with valid_loss value: 0.6442688703536987.
Better model found at epoch 7 with valid_loss value: 0.519091784954071.
Better model found at epoch 8 with valid_loss value: 0.407736212015152.
Better model found at epoch 12 with valid_loss value: 0.4070434272289276.
Better model found at epoch 14 with valid_loss value: 0.35599657893180847.
Better model found at epoch 21 with valid_loss value: 0.33475297689437866.
Better model found at epoch 37 with valid_loss value: 0.33224350214004517.
Better model found at epoch 38 with valid_loss value: 0.3321360647678375.
set state called


The best model is saved in `{PATH}/models/best_a40.pth`, unless another filename has been specified.

## Experimental results
    
    Results (split: isolated, 1cycle)
                                           
      n_epoch  max_lr  arch  bs    Val loss  TTA loss    time
         40     3e-2    18   32      0.342     0.350     6:27
         40     6e-2    18   32      0.328     0.331     6:43
         40    10e-2    18   32      0.324     0.324     6:43
         40     6e-2    18   64      0.338     0.344     6:13
         40    10e-2    18   64      0.331     0.331     6:10

         40     3e-2    34   32      0.338     0.344     9:20
         40     6e-2    34   32      0.327     0.325     9:44
         40    10e-2    34   32      0.328     0.326     9:50
         40    20e-2    34   32      0.319     0.316     9:38
         40    40e-2    34   32      0.331     0.329     9:40
         40     3e-2    34   64      0.337     0.336     8:25
         40     6e-2    34   64      0.322     0.324     8:25
       ★ 40    10e-2    34   64      0.315     0.313     8:37
         40    20e-2    34   64      0.322     0.322     8:30
         40    30e-2    34   64      0.322     0.319     8:20

        100    30e-2    34   64      0.327     0.324    20:35

    Results (split: isolated, flat+cosine)

         40     6e-2    34   64      0.332     0.334     8:00
         40    10e-2    34   64      0.334     -----     7:52                 

    Results (split: random, 1cycle)

      n_epoch  max_lr  arch  bs    Val loss  TTA loss    time
         40    10e-2    18   32      0.324     0.326     6:51

         40    10e-2    34   64      0.335     0.333     8:37
         40    30e-2    34   64      0.334     0.330     8:50
         


        

# Evaluate

In [4]:
context = "environment"    
# context = "random"
learn.load(f'best_xGASS_{context}-34_flatcosine');

NameError: ignored

In [6]:
learn.to_fp32();
fgas_pred, fgas_true = learn.TTA(scale=1.)
root_mean_squared_error(fgas_pred, fgas_true)

tensor(0.3335)

# Plot

In [0]:
results = pd.DataFrame({'fgas_pred': fgas_pred.squeeze(), 'fgas_true': fgas_true.squeeze()})
results['GASS'] = np.array([os.path.basename(fn).strip('.jpg') for fn in data.valid_ds.items])
results.set_index('GASS', drop=True, inplace=True)


In [0]:

plt.figure(figsize=(6.2, 5), dpi=150)
plt.hist2d(results.fgas_true, results.fgas_pred, cmap='Reds', range=[(-2.5, 1.), (-2.5, 1.)], bins=21);
plt.plot([-2.5, 2.5], [-2.5, 2.5], c='black', ls='-', lw=2)
plt.gca().set_aspect('equal')
plt.colorbar();
plt.xlabel(r'$\mu_{\rm true}$', fontsize=12)
plt.ylabel(r'$\mu_{\rm pred}$', fontsize=12);
plt.xticks(np.arange(-2.5, 1.5, .5))
plt.yticks(np.arange(-2.5, 1.5, .5));

plt.text(-2.3, 0.5, f'{context} \n' + f'RMSE = {root_mean_squared_error(fgas_pred, fgas_true):.2f}', fontsize=12);

# terrible terrible way to determine which catalog we're using
catalog_id = "B" if len(df) == 4797 else "A"
plt.savefig(f'{PATH}/results/figures/results-hist_xGASS-{context}')